In [1]:
import pyodbc
import pandas as pd
from time import gmtime, strftime
from alpha_vantage.timeseries import TimeSeries
import time
import QuantLib as ql
from utilities.date_utilities import *


In [2]:
connection = pyodbc.connect('Driver=SQL Server;'
                      'Server=DESKTOP-0CQQR1E\SQLEXPRESS;'
                      'Database=securities_master;'
                      'Trusted_Connection=yes;')
cursor=connection.cursor()
ts = TimeSeries(key='NHXLC1G323WY6E6I', output_format='pandas')

In [3]:

def check_update(ticker):
    """
    check if a  ticker's history is None or the latest update is performed more (excluding) than 1 data before

    :param ticker:
    :return: -999 for not None, float for business days since last update
    """
    global connection
    proceed_update=False
    
    

    
    data = pd.read_sql_query(
        f"select top 1 *  from securities_hist_rs2000 where ticker='{ticker}' order by eff_date desc", connection)

    if len(data)<1: # first-time update
        return -999,None
    else:
        current_time = pd.to_datetime(strftime("%Y-%m-%d %H:%M:%S", gmtime()))
        last_updated_time = data["eff_date"][0]
        business_day_since_update = business_day_between(last_updated_time,current_time)
        return business_day_since_update,data["eff_date"][0]
    





In [4]:


def insert_sec_full(ticker,sec_id,sleep_time=14):
    global connection,cursor,ts
    b_days_since_last_eff, last_eff = check_update(ticker) 

    if b_days_since_last_eff==-999: # security is new, loading all data
        print(f"{ticker} does not exist in the database, fetching full history...")
        data = ts.get_daily_adjusted(ticker, outputsize='full')[0]
  
    elif b_days_since_last_eff>=10: # security exists but not up to date
        print(f"{ticker} is {b_days_since_last_eff} days behind, fetching new data...")
        data = ts.get_daily_adjusted(ticker, outputsize='full')[0]
        data = data[:last_eff].iloc[:-1]
        time.sleep(sleep_time)
    else:
        print(f"{ticker} is up to date, no action needed.")
        return 
    time.sleep(sleep_time)
    
    if len(data)>0:
        
        
        for idx in range(len(data)):
            record=data.iloc[idx]
            curr_time=strftime("%Y-%m-%d %H:%M:%S", gmtime())
            insert_str = f"""
            INSERT INTO securities_hist_rs2000
            (
            sec_id,
            ticker,
            instrument,
            eff_date,
            open_price,
            high_price,
            low_price,
            close_price,
            adj_close_price,
            volume,
            dividend,
            split,
            created_date,
            last_updated_date)
            values (

            '{sec_id}',
            '{ticker}',
            '{'Common Stock'}',
            '{record.name}',
            {record['1. open']},
            {record['2. high']},
            {record['3. low']},
            {record['4. close']},
            {record['5. adjusted close']},
            {record['6. volume']},
            {record['7. dividend amount']},
            {record['8. split coefficient']},
            '{curr_time}',
            '{curr_time}'

            )
            """

            cursor.execute(insert_str)
            connection.commit()
        print(f"{ticker} updated.")


In [5]:
static_sec_info = pd.read_sql_query("select * from securities_rs2000", connection)


In [12]:


static_sec_info = pd.read_sql_query("select * from securities_rs2000", connection)


# update tickers

for  ticker,sec_id in zip(sorted(static_sec_info.ticker.values)[1505:],sorted(static_sec_info.sec_id.values)):
    try:

        insert_sec_full(ticker,sec_id)
    except Exception:
        
        print(f"fatal error {ticker}")
        time.sleep(15)
        break

        


connection.close()



RMNI is up to date, no action needed.
RMR is up to date, no action needed.
RMTI is up to date, no action needed.
RNST is up to date, no action needed.
ROAD does not exist in the database, fetching full history...
ROAD updated.
ROCK does not exist in the database, fetching full history...
ROCK updated.
ROG does not exist in the database, fetching full history...
ROG updated.
ROIC does not exist in the database, fetching full history...
ROIC updated.
ROLL does not exist in the database, fetching full history...
ROLL updated.
RPAI does not exist in the database, fetching full history...
RPAI updated.
RPAY does not exist in the database, fetching full history...
RPAY updated.
RPD does not exist in the database, fetching full history...
RPD updated.
RPT does not exist in the database, fetching full history...
RPT updated.
RRBI does not exist in the database, fetching full history...
RRBI updated.
RRC does not exist in the database, fetching full history...
RRC updated.
RRGB does not exist i

SM updated.
SMBC does not exist in the database, fetching full history...
SMBC updated.
SMBK does not exist in the database, fetching full history...
SMBK updated.
SMCI does not exist in the database, fetching full history...
SMCI updated.
SMED does not exist in the database, fetching full history...
SMED updated.
SMMF does not exist in the database, fetching full history...
SMMF updated.
SMP does not exist in the database, fetching full history...
SMP updated.
SMPL does not exist in the database, fetching full history...
SMPL updated.
SMSI does not exist in the database, fetching full history...
SMSI updated.
SMTC does not exist in the database, fetching full history...
SMTC updated.
SNBR does not exist in the database, fetching full history...
SNBR updated.
SNCR does not exist in the database, fetching full history...
SNCR updated.
SNDX does not exist in the database, fetching full history...
SNDX updated.
SNFCA does not exist in the database, fetching full history...
SNFCA updated.


TGH updated.
TGI does not exist in the database, fetching full history...
TGI updated.
TGNA does not exist in the database, fetching full history...
TGNA updated.
TGTX does not exist in the database, fetching full history...
TGTX updated.
TH does not exist in the database, fetching full history...
TH updated.
THC does not exist in the database, fetching full history...
THC updated.
THFF does not exist in the database, fetching full history...
THFF updated.
THR does not exist in the database, fetching full history...
THR updated.
THRM does not exist in the database, fetching full history...
THRM updated.
TILE does not exist in the database, fetching full history...
TILE updated.
TIPT does not exist in the database, fetching full history...
TIPT updated.
TISI does not exist in the database, fetching full history...
TISI updated.
TITN does not exist in the database, fetching full history...
TITN updated.
TLYS does not exist in the database, fetching full history...
TLYS updated.
TMDX does

VECO updated.
VERI does not exist in the database, fetching full history...
VERI updated.
VERO does not exist in the database, fetching full history...
VERO updated.
VERU does not exist in the database, fetching full history...
VERU updated.
VG does not exist in the database, fetching full history...
VG updated.
VGR does not exist in the database, fetching full history...
VGR updated.
VHC does not exist in the database, fetching full history...
VHC updated.
VIAV does not exist in the database, fetching full history...
VIAV updated.
VICR does not exist in the database, fetching full history...
VICR updated.
VIE does not exist in the database, fetching full history...
VIE updated.
VIR does not exist in the database, fetching full history...
VIR updated.
VIVO does not exist in the database, fetching full history...
VIVO updated.
VKTX does not exist in the database, fetching full history...
VKTX updated.
VLGEA does not exist in the database, fetching full history...
VLGEA updated.
VLY does

ZIXI updated.
ZNTL does not exist in the database, fetching full history...
ZNTL updated.
ZUMZ does not exist in the database, fetching full history...
ZUMZ updated.
ZUO does not exist in the database, fetching full history...
ZUO updated.
ZYXI does not exist in the database, fetching full history...
ZYXI updated.


In [ ]:
connection.close()